In [6]:
def insert_covid_data(df, mycursor, mydb):

    for row in df.itertuples(index=False):

        mycursor.execute('''
        INSERT INTO covid_data_total VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            row.location, row.date, row.total_cases, row.new_cases,
            row.total_deaths, row.new_deaths, row.reproduction_rate,
            row.icu_patients, row.hosp_patients, row.total_tests,
            row.new_tests, row.tests_per_case, row.total_vaccinations,
            row.people_vaccinated, row.people_fully_vaccinated,
            row.population_density, row.median_age, row.aged_65_older,
            row.aged_70_older, row.gdp_per_capita, row.extreme_poverty,
            row.handwashing_facilities, row.life_expectancy, row.population,
            row.PCA1, row.PCA2, row.PCA3, row.PCA4, row.Cluster
        ))

        mydb.commit()         

In [11]:
def insert_covid_data_lote(df, mycursor, mydb, batch_size):
    rows = []
    for row in df.itertuples(index=False):
        rows.append((
            row.location, row.date, row.total_cases, row.new_cases,
            row.total_deaths, row.new_deaths, row.reproduction_rate,
            row.icu_patients, row.hosp_patients, row.total_tests,
            row.new_tests, row.tests_per_case, row.total_vaccinations,
            row.people_vaccinated, row.people_fully_vaccinated,
            row.population_density, row.median_age, row.aged_65_older,
            row.aged_70_older, row.gdp_per_capita, row.extreme_poverty,
            row.handwashing_facilities, row.life_expectancy, row.population,
            row.PCA1, row.PCA2, row.PCA3, row.PCA4, row.Cluster
        ))
        
        # Insere os dados em lote quando atinge o tamanho do batch
        if len(rows) == batch_size:
            mycursor.executemany('''
            INSERT INTO covid_data_total VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', rows)
            mydb.commit()
            rows.clear()  # Limpa a lista para o próximo lote

    # Insere o restante dos dados que não completaram um batch
    if rows:
        mycursor.executemany('''
        INSERT INTO covid_data_total VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', rows)
        mydb.commit()

import mysql.connector
import pandas as pd

# Carregar os dados
df = pd.read_csv('dados_com_clusters.csv')

try:  
    mydb = mysql.connector.connect(
        user="admin",
        password="urubu100",
        host="database-anti-plague.c5efil1wob4a.us-east-1.rds.amazonaws.com",
        database="AntiPlague"
    )

    batch_size=50000

    if mydb.is_connected():
        mycursor = mydb.cursor()
        insert_covid_data_lote(df, mycursor, mydb, batch_size)  # Tamanho do lote ajustável
        
except mysql.connector.Error as e:
    print("Erro ao conectar com o MySQL", e)
finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()


In [7]:
import mysql.connector
import pandas as pd

# Carregar os dados
df = pd.read_csv('dados_com_clusters.csv')

try:  
    mydb = mysql.connector.connect(
        user= "admin",
        password= "urubu100",
        host= "database-anti-plague.c5efil1wob4a.us-east-1.rds.amazonaws.com",
        database= "AntiPlague"
    )

    if mydb.is_connected():
        mycursor = mydb.cursor()
        insert_covid_data(df, mycursor, mydb)   
        
except mysql.connector.Error as e:
    print("Erro ao conectar com o MySQL", e)
finally:
    if(mydb.is_connected()):
        mycursor.close()
        mydb.close()

KeyboardInterrupt: 